In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta, date
import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import matplotlib.patches as patches
from matplotlib.colors import TwoSlopeNorm
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from operator import itemgetter
from mapping_plot_attributes import *
from mapping_tickers import *
from utils import *
from download_data import DownloadData

In [2]:
tickers = list(magnificent_7_tickers.keys())
end_date = datetime.today()
hist_years, hist_months, hist_days = 1, 0, 0
start_date = datetime(end_date.year - hist_years, end_date.month - hist_months, end_date.day - hist_days)
tk_market = '^GSPC'

hist_data = DownloadData(end_date, start_date, tickers, tk_market)

downloaded_data = hist_data.download_yh_data(start_date, end_date, tickers, tk_market)
df_adj_close = downloaded_data['Adj Close']
df_close = downloaded_data['Close']
dict_ohlc = downloaded_data['OHLC']

tk = 'AAPL'
df_ohlc = dict_ohlc[tk]
ohlc_tk = df_ohlc.copy()
adj_close_tk = df_adj_close[tk]
close_tk = df_close[tk]
open_tk = ohlc_tk['Open']
high_tk = ohlc_tk['High']
low_tk = ohlc_tk['Low']

price_type_map = {
    'Adj Close': adj_close_tk,
    'Adjusted Close': adj_close_tk,
    'Close': close_tk,
    'Open': open_tk,
    'High': high_tk,
    'Low': low_tk
}

# display(df_adj_close)
# display(df_close)
# display(df_ohlc)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

The portfolio data will be truncated to end at the latest available date of 2024-09-27.


In [3]:
def weighted_mean(values):
    """
    values: a list, tuple or series of numerical values
    """
    if isinstance(values, (list, tuple)):
        values = pd.Series(values)
    
    n = len(values)
    weight_sum = n * (n + 1) / 2
    weights = range(n + 1)[1:]
    wm = values @ weights / weight_sum
    return wm

In [4]:
def wilder_moving_average(df_tk, n):
    """
     J. Welles Wilder's EMA 
     https://stackoverflow.com/questions/40256338/calculating-average-true-range-atr-on-ohlc-data-with-python
    """
    wwma = df_tk.ewm(alpha = 1 / n, adjust = False).mean()
    return wwma

In [5]:
def average_true_rate(
    close_tk,
    high_tk,
    low_tk,
    n = 14
):
    """
    https://stackoverflow.com/questions/40256338/calculating-average-true-range-atr-on-ohlc-data-with-python
    
    """
    tr_cols = ['tr0', 'tr1', 'tr2']
    df_tr = pd.DataFrame(columns = tr_cols, index = close_tk.index)

    df_tr['tr0'] = abs(high_tk - low_tk)
    df_tr['tr1'] = abs(high_tk - close_tk.shift())
    df_tr['tr2'] = abs(low_tk - close_tk.shift())
    tr = df_tr[tr_cols].max(axis = 1)

    atr = wilder_moving_average(tr, n)
    atrp = atr / close_tk * 100
    atr_data = {
        'atr': atr,
        'atrp': atrp
    }
    
    return atr_data

In [6]:
atr_data = average_true_rate(close_tk, high_tk, low_tk)
atr = atr_data['atr']
atrp = atr_data['atrp']
display(atr)
print(atr.max())

Date
2023-09-28    4.410004
2023-09-29    4.290004
2023-10-02    4.224290
2023-10-03    4.131841
2023-10-04    4.068138
                ...   
2024-09-23    5.072198
2024-09-24    4.968471
2024-09-25    4.852865
2024-09-26    4.726946
2024-09-27    4.547878
Length: 252, dtype: float64

6.9204888578926465


In [7]:
def moving_average(
    df_tk,
    ma_type,
    ma_window,
    min_periods = 1
):
    """
    df_tk:      
        a series of price values, taken as a column of df_close or df_adj_close for ticker tk
    ma_type:    
        simple ('sma'),
        exponential ('ema'),
        double exponential ('dema'),
        triple exponential ('tema'),
        weighted ('wma')
    window:
        length in days
    Returns ma
    """

    if not isinstance(df_tk, pd.Series):
        print('Incorrect format of input data')
        exit
    
    if ma_type in ['ema', 'dema', 'tema']:
        ma = df_tk.ewm(span = ma_window).mean()
        if ma_type in ['dema', 'tema']:
            ma = ma.ewm(span = ma_window).mean()
            if ma_type == 'tema':
                ma = ma.ewm(span = ma_window).mean()
    
    elif ma_type == 'wma':
        ma = df_tk.rolling(window = ma_window, min_periods = min_periods).apply(lambda x: weighted_mean(x))

    else:  # 'sma' or anything else
        ma = df_tk.rolling(window = ma_window, min_periods = min_periods).mean()
    
    return ma

In [8]:
wma = moving_average(close_tk, 'wma', 14)
wwma = wilder_moving_average(close_tk, 14)
display(wma-wwma)

Date
2023-09-28    0.000000
2023-09-29    0.309526
2023-10-02    1.450272
2023-10-03    1.348869
2023-10-04    1.583570
                ...   
2024-09-23    0.137153
2024-09-24    0.507975
2024-09-25    0.777861
2024-09-26    1.097252
2024-09-27    1.390563
Name: AAPL, Length: 252, dtype: float64

In [9]:
ma_list = [
    {
        'ma_idx': 1,
        'ma_type': 'sma',
        # 'ma_window': 10,
        'ma_window': 5,
        'showlegend': True
    },
    {
        'ma_idx': 2,
        'ma_type': 'sma',
        # 'ma_window': 20,
        'ma_window': 10,
        'showlegend': True
    },
    {
        'ma_idx': 3,
        'ma_type': 'sma',
        # 'ma_window': 30,
        'ma_window': 15,
        'showlegend': True
    },
    {
        'ma_idx': 4,
        'ma_type': 'sma',
        # 'ma_window': 40,
        'ma_window': 20,
        'showlegend': True
    }
    ,
    {
        'ma_idx': 5,
        'ma_type': 'sma',
        #'ma_window': 50,
        'ma_window': 25,
        'showlegend': True
    },
    {
        'ma_idx': 6,
        'ma_type': 'sma',
        # 'ma_window': 60,
        'ma_window': 30,
        'showlegend': True
    }
]

ema_list = [
    {
        'ma_idx': 1,
        'ma_type': 'ema',
        'ma_window': 10,
        'showlegend': True
    },
    {
        'ma_idx': 2,
        'ma_type': 'ema',
        'ma_window': 20,
        'showlegend': True
    },
    {
        'ma_idx': 3,
        'ma_type': 'ema',
        'ma_window': 30,
        'showlegend': True
    },
    {
        'ma_idx': 4,
        'ma_type': 'ema',
        'ma_window': 40,
        'showlegend': True
    }
    ,
    {
        'ma_idx': 5,
        'ma_type': 'ema',
        'ma_window': 50,
        'showlegend': True
    },
    {
        'ma_idx': 6,
        'ma_type': 'ema',
        'ma_window': 60,
        'showlegend': True
    }
]

In [10]:
def add_overlay(
    fig_data,
    df,
    name,
    color_idx,
    showlegend = True,
    legendgroup = 'upper',
    theme = 'dark',
    color_theme = 'gold'
):
    """
    fig_data: a dictionary of the underlying figure data

    y_min_fig: y_min on the existing fig
    y_max_fig: y_max on the existing fig
    color_idx: an integer (0, ...) indicating the color from those available in theme_style
    showlegend: whether or not to show line in legend (e.g. we only need one Bollinger band in legend)
    legendgroup: 'upper' is top graph in subplots (row 1), 'lower' is the stacked lower graph (row 2)

    Returns the updated fig_data dictionary
    """

    subplot_row = legendgroup_map[legendgroup]

    style = theme_style[theme]
    overlay_colors = style['overlay_color_theme'][color_theme]

    fig = fig_data['fig']
    y_min_fig = fig_data['y_min']
    y_max_fig = fig_data['y_max']

    min_y = min(df)
    max_y = max(df)
    y_min, y_max = set_axis_limits(min_y, max_y)

    new_y_min, new_y_max = min(y_min, y_min_fig), max(y_max, y_max_fig)

    if color_idx >= len(overlay_colors):
        # Take the last overlay color from the available list
        color_idx = -1

    fig.add_trace(
        go.Scatter(
            x = df.index.astype(str),
            y = df,
            line = dict(color = overlay_colors[color_idx]),
            name = name,
            showlegend = showlegend,
            legendgroup = legendgroup
        ),
        row = subplot_row, col = 1    
    )

    fig.update_yaxes(
        range = (new_y_min, new_y_max),
        gridcolor = style['y_gridcolor'],
        ticks = 'outside',
        ticklen = 8,
        row = subplot_row, col = 1
    )

    fig_data = {
        'fig': fig,
        'y_min': new_y_min,
        'y_max': new_y_max
    }

    return fig_data

In [45]:
def add_ma_overlays(
    fig_data,
    df_price,
    ma_list,
    x_min = None,
    x_max = None,
    legendgroup = 'upper',
    theme = 'dark',
    color_theme = 'gold'
):
    """
    df_price: 
        df_close or df_adj_close, depending on the underlying figure
    legendgroup:
        'upper' is top graph in subplots (row 1), 'lower' is the stacked lower graph (row 2)
    ma_list: 
        list of ma overlay dictionaries, containing
         - ma_idx ma index (1, 2,...)
         - ma_type: 'sma' (default), 'ema', 'dema', 'tema' or 'wma'
         - ma_window, in days
         - showlegend: include in plot legend or not
    """

    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max

    n_ma = len(ma_list)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_ma]

    current_names = [trace['name'] for trace in fig_data['fig']['data']]
    
    ma_overlays = []

    for i, ma in enumerate(ma_list):
        
        ma_type = ma['ma_type']
        ma_window = ma['ma_window']
        ma_name = f'{ma_type.upper()} {ma_window}'

        if ma_name not in current_names:
    
            ma_data = moving_average(
                df_price[x_min: x_max],
                ma_type,
                ma_window
            )
            ma_color_idx = overlay_color_idx[i]
            ma_showlegend = ma['showlegend']

            ma_overlays.append({
                'data': ma_data,
                'name': ma_name,
                'color_idx': ma_color_idx,
                'showlegend': ma_showlegend
            })

    color_map = {}

    for overlay in ma_overlays:
        fig_data = add_overlay(
            fig_data,
            overlay['data'],
            overlay['name'],
            overlay['color_idx'],
            overlay['showlegend'],
            legendgroup = legendgroup,
            theme = theme,
            color_theme = color_theme
        )        
        color_map.update({overlay['name']: overlay['color_idx']})

    n_traces_upper = len([x for x in fig_data['fig']['data'] if x['legendgroup'] == 'upper'])
    n_traces_lower = len([x for x in fig_data['fig']['data'] if x['legendgroup'] == 'lower'])

    legend_gap = 631.6 - 19 * n_traces_upper

    # Update layout and axes
    fig_data['fig'].update_layout(
        legend_tracegroupgap = legend_gap
    )

    fig_data.update({'color_map': color_map})
    fig_data.update({
        'n_traces': {
            'upper': n_traces_upper,
            'lower': n_traces_lower,
        }
    })

    return fig_data

In [12]:
def plot_hist_plotly(
    df_price,
    tk,
    n_ticks_max = 48,
    n_yticks_max = 16,
    plot_width = 1450,
    plot_height = 750,
    title_font_size = 32,
    theme = 'dark',
    price_type = 'adjusted close'
):
    """
    price_type: one of ['adjusted close', 'close', 'open', 'high', 'low']

    """

    if isinstance(df_price, pd.Series):
        df_tk = df_price.copy()
    elif isinstance(df_price, pd.DataFrame):
        df_tk = df_price[tk]
    else:
        print('Incorrect format of input data')
        exit

    style = theme_style[theme]

    min_y = min(df_tk)
    max_y = max(df_tk)
    y_min, y_max = set_axis_limits(min_y, max_y)

    fig = make_subplots(rows = 1, cols = 1)

    fig.add_trace(
        go.Scatter(
            x = df_tk.index.astype(str),
            y = df_tk,
            line = dict(color = style['basecolor']),
            showlegend = True,
            legendgroup = 'upper',            
            name = price_type.title()
        )
    )
    # Add plot border
    fig.add_shape(
        type = 'rect',
        xref = 'paper',
        yref = 'paper',
        x0 = 0,
        x1 = 1,
        y0 = 0,
        y1 = 1,
        line_color = style['x_linecolor'],
        line_width = 0.3
    )
    # Update layout and axes
    fig.update_layout(
        width = plot_width,
        height = plot_height,
        xaxis_rangeslider_visible = False,
        template = style['template'],
        legend_groupclick = 'toggleitem',        
        yaxis_title = f'{price_type.title()}',
        margin_t = 60,
        title = dict(
            text = f'{tk} {price_type.title()}',
            font_size = title_font_size,
            y = 0.975,
            x = 0.45,
            xanchor = 'center',
            yanchor = 'top'
        )
    )
    fig.update_xaxes(
        type = 'category',
        gridcolor = style['x_gridcolor'],        
        nticks = n_ticks_max,
        tickangle = -90,
        ticks = 'outside',
        ticklen = 8        
    )
    fig.update_yaxes(
        range = (y_min, y_max),
        gridcolor = style['y_gridcolor'],
        nticks = n_yticks_max,
        ticks = 'outside',
        ticklen = 8
    )

    fig_data = {
        'fig': fig,
        'y_min': y_min,
        'y_max': y_max
    }

    return fig_data

In [13]:
def macd(
    df_tk,
    signal_window = 9      
):
    """
    df_tk: a series of price values, taken as a column of df_close or df_adj_close for ticker tk
    
    """ 

    if not isinstance(df_tk, pd.Series):
        print('Incorrect format of input data')
        exit
    
    ema_26 = df_tk.ewm(span = 26).mean()
    ema_12 = df_tk.ewm(span = 12).mean()
    macd_line = ema_12 - ema_26
        
    macd_signal = macd_line.ewm(span = signal_window).mean()

    macd_data = {
        'MACD': macd_line,
        'MACD Signal': macd_signal,
        'MACD Signal Window': signal_window
    }

    return macd_data

In [51]:
def add_macd_stacked(
    fig_data,
    tk_macd,
    macd_data,
    n_yticks_max = 11,
    legendgroup = 'lower',
    theme = 'dark'
):
    """
    MACD with a signal line added to the lower stacked plot
    """
    
    # x_min = start_date if x_min is None else x_min
    # x_max = end_date if x_max is None else x_max

    fig_macd = fig_data['fig']

    style = theme_style[theme]
    
    subplot_row = legendgroup_map[legendgroup]
        
    macd = macd_data['MACD']
    macd_signal = macd_data['MACD Signal']
    macd_signal_window = macd_data['MACD Signal Window']
 
    # x_min = str(macd.index.min().date())
    # x_max = str(macd.index.max().date())
    # print(x_min, x_max)

    min_macd = min(min(macd), min(macd_signal))
    max_macd = max(max(macd), max(macd_signal))
    y_macd_min, y_macd_max = set_axis_limits(min_macd, max_macd, max_n_intervals = 8)
    y_macd_max *= 0.999
    
    macd_positive = macd.copy()
    macd_positive.iloc[np.where(macd_positive < 0)] = np.nan
    macd_negative = macd.copy()
    macd_negative.iloc[np.where(macd_negative >= 0)] = np.nan
    
    title_macd = f'{tk_macd} Moving Average Convergence Divergence (EMA 12-26)'

    fig_macd.add_trace(
        go.Bar(
            x = macd_positive.index.astype(str),
            y = macd_positive,
            marker_color = style['green_color'],
            width = 1,
            name = 'MACD > 0',
            legendgroup = legendgroup
        ),
        row = subplot_row, col = 1
    )
    fig_macd.add_trace(
        go.Bar(
            x = macd_negative.index.astype(str),
            y = macd_negative,
            marker_color = style['red_color'],
            width = 1,
            name = 'MACD < 0',
            legendgroup = legendgroup
        ),
        row = subplot_row, col = 1
    )

    fig_macd.add_trace(
        go.Scatter(
            x = macd_signal.index.astype(str),
            y = macd_signal,
            line = dict(color = style['signal_color']),
            # name = 'Signal Line'  # 9-day span is a standard, no need to customise it
            name = f'EMA {macd_signal_window} Signal',
            legendgroup = legendgroup
        ),
        row = subplot_row, col = 1
    )

    n_traces_upper = len([x for x in fig_macd['data'] if x['legendgroup'] == 'upper'])
    n_traces_lower = len([x for x in fig_macd['data'] if x['legendgroup'] == 'lower'])
    legend_gap = 631.6 - 19 * n_traces_upper

    # Update layout and axes
    fig_macd.update_layout(
        legend_tracegroupgap = legend_gap
    )

    fig_macd.update_yaxes(
        title_text = f'MACD',
        range = (y_macd_min, y_macd_max),
        gridcolor = style['y_gridcolor'],
        nticks = n_yticks_max,
        ticks = 'outside',
        ticklen = 8,
        row = subplot_row, col = 1
    )

    fig_macd_data = {
        'fig': fig_macd,
        'y_min': fig_data['y_min'],
        'y_max': fig_data['y_max'],
        'y_min_lower': y_macd_min,
        'y_max_lower': y_macd_max,
        'n_traces': {
            'upper': n_traces_upper,
            'lower': n_traces_lower,
        }
    }

    return fig_macd_data 

In [41]:
def plot_stacked_plotly(
    df_main,
    df_stacked,
    tk,
    n_ticks_max = 48,
    n_yticks_max_upper = 16,
    # n_yticks_max_lower = 9,
    plot_width = 1450,
    plot_height_upper = 750,
    plot_height_lower = 250,
    add_main_title = True,
    title_main_font_size = 32,
    price_type = 'close',
    stacked_label = '',
    theme = 'dark'
):
    """
    price_type: one of ['adjusted close', 'close', 'open', 'high', 'low']

    """

    if isinstance(df_main, pd.Series):
        df_tk = df_main.copy()
    elif isinstance(df_main, pd.DataFrame):
        df_tk = df_main[tk]
    else:
        print('Incorrect format of input data')
        exit

    if isinstance(df_stacked, pd.Series):
        stacked_tk = df_stacked.copy()
    elif isinstance(df_stacked, pd.DataFrame):
        stacked_tk = df_stacked[tk]
    else:
        print('Incorrect format of input data')
        exit

    style = theme_style[theme]

    x_min = str(min(df_tk.index).date())
    x_max = str(max(df_tk.index).date())

    min_y_upper = min(df_tk)
    max_y_upper = max(df_tk)
    y_min_upper, y_max_upper = set_axis_limits(min_y_upper, max_y_upper)  # default = 9

    min_y_lower = min(stacked_tk)
    max_y_lower = max(stacked_tk)
    y_min_lower, y_max_lower = set_axis_limits(min_y_lower, max_y_lower, max_n_intervals = 9)

    subplot_vertical_spacing = 0
    
    plot_height_total = plot_height_upper + plot_height_lower
    height_pct_upper = plot_height_upper / plot_height_total
    height_pct_lower = plot_height_lower / plot_height_total

    title_main = f'{tk} {price_type.title()}'

    fig = make_subplots(
        rows = 2,
        cols = 1,
        shared_xaxes = True,
        vertical_spacing = subplot_vertical_spacing,
        row_heights = [
            height_pct_upper,
            height_pct_lower
        ]
    )

    fig.add_trace(
        go.Scatter(
            x = df_tk.index.astype(str),
            y = df_tk,
            line_color = style['basecolor'],
            name = f'{price_type.title()}',
            showlegend = True,
            legendgroup = 'upper'
        ),
        row = 1, col = 1
    )

    # Add a dummy
    fig.add_trace(
        go.Scatter(
            x = df_tk.index.astype(str),
            y = df_tk,
            line_width = 0,
            showlegend = False,
            legendgroup = 'dummy'
        ),
        row = 2, col = 1
    )

    # Count traces in grouplegend 'upper'
    # Need to update the count after adding overlays to upper graph
    # n_traces_upper = len([x for x in fig['data'] if x['legendgroup'] == 'upper'])

    # n_traces_upper = 8

    # Found from linear regression of best-looking test legendgroup_gap values
    # This likely depends on the top margin (60), the size of the legend title font (16)
    # and perhaps also on the width of the date labels, absent from the appuer plot.
    # legendgroup_gap = 613.8 - 19 * n_traces_upper  # with legend titles

    # legendgroup_gap = 637.1 - 19 * n_traces_upper  # without legend titles, lower plot_height = 300

    # legendgroup_gap = 631.6 - 19 * n_traces_upper  # without legend titles, lower plot_height = 250

    # print(n_traces_upper, legendgroup_gap)

    # Add plot border
    fig.add_shape(
        type = 'rect',
        xref = 'x',  # use 'x' because 'paper' does not work correctly with stacked plots
        yref = 'paper',
        x0 = x_min,
        x1 = x_max,
        y0 = 0,
        y1 = (1 - subplot_vertical_spacing) * height_pct_lower,
        line_color = style['x_linecolor'],
        line_width = 1
    )
    fig.add_shape(
        type = 'rect',
        xref = 'x',  # use 'x' because 'paper' does not work correctly with stacked plots
        yref = 'paper',
        x0 = x_min,
        x1 = x_max,
        y0 = 1 - (1 - subplot_vertical_spacing) * height_pct_upper,
        y1 = 1,
        line_color = style['x_linecolor'],
        line_width = 1
    )

    if subplot_vertical_spacing == 0:
        y_max_lower *= 0.999  # to avoid overlapping with y_min_main label

    # Update layout and axes
    fig.update_layout(
        width = plot_width,
        height = plot_height_total * (1 + subplot_vertical_spacing),
        xaxis_rangeslider_visible = False,
        template = style['template'],
        legend_groupclick = 'toggleitem'
        # legend_tracegroupgap = legendgroup_gap
        # legend_tracegroupgap = 18
    )
    if add_main_title:
        fig.update_layout(
            margin_t = 60,
            title = dict(
                text = title_main,
                font_size = title_main_font_size,
                y = 0.98,
                x = 0.45,
                xanchor = 'center',
                yanchor = 'top'
            )
        )

    fig.update_xaxes(
        type = 'category',
        gridcolor = style['x_gridcolor'],
        nticks = n_ticks_max,
        tickangle = -90,
        ticks = 'outside',
        ticklen = 8,
        row = 1, col = 1
    )
    fig.update_xaxes(
        type = 'category',
        gridcolor = style['x_gridcolor'],
        nticks = n_ticks_max,
        tickangle = -90,
        ticks = 'outside',
        ticklen = 8,
        row = 2, col = 1
    )
    fig.update_yaxes(
        range = (y_min_upper, y_max_upper),
        title = price_type.title(),
        gridcolor = style['y_gridcolor'],
        nticks = n_yticks_max_upper,
        ticks = 'outside',
        ticklen = 8,
        row = 1, col = 1
    )
    fig.update_yaxes(
        range = (y_min_lower, y_max_lower),
        title = stacked_label,
        showgrid = True,
        gridcolor = style['y_gridcolor'],
        # nticks = n_yticks_max_lower,
        ticks = 'outside',
        ticklen = 8,
        row = 2, col = 1
    )

    """
    fig.add_annotation(
        # xref = 'paper',  # this has no effect because x is categorical
        xref = 'x',
        yref = 'y',
        x = half_length,  # the closest number of the "category" from all x values
        y = y_max_main - 0.8 * title_stacked_font_size / plot_height_total,
        xanchor = 'center',
        yanchor = 'top',
        text = f'{price_type.title()}',
        # font_size = title_main_font_size,
        font_size = title_stacked_font_size,
        showarrow = False,
        bgcolor = fig['layout']['template']['layout']['plot_bgcolor'],
        borderwidth = 1,
        borderpad = 3,
        bordercolor = style['x_linecolor'],
        row = 1, col = 1
    )
    fig.add_annotation(
        # xref = 'paper',  # this has no effect because x is categorical
        xref = 'x2',
        yref = 'y2',
        x = half_length,  # the closest number of the "category" from all x values
        y = y_max_stacked - 0.8 * title_stacked_font_size / plot_height_total,
        xanchor = 'center',
        yanchor = 'top',
        text = stacked_label,
        font_size = title_stacked_font_size,
        showarrow = False,
        bgcolor = fig['layout']['template']['layout']['plot_bgcolor'],
        borderwidth = 1,
        borderpad = 3,
        bordercolor = style['x_linecolor'],
        row = 2, col = 1
    )
    """

    fig_data = {
        'fig': fig,
        'y_min': y_min_upper,
        'y_max': y_max_upper,
        'y_min_lower': y_min_lower,
        'y_max_lower': y_max_lower
    }

    return fig_data

In [16]:
macd_data = macd(adj_close_tk)

In [54]:
tk = 'AAPL'
theme = 'dark'
color_theme = 'magenta'
color_theme_2 = 'gold'

print(str(min(close_tk.index)))

fig_close_data = plot_hist_plotly(close_tk, tk, price_type = 'close', theme = theme)
fig_close = fig_close_data['fig']
# fig_close.show()

sma30 = moving_average(close_tk, 'sma', 30)
macd = macd_data['MACD']
macd_signal = macd_data['MACD Signal']

fig_data = plot_stacked_plotly(close_tk, macd, tk, price_type = 'close', stacked_label = 'MACD', theme = theme)
# fig_data = add_ma_overlays(fig_data, close_tk, ma_list[: 6], legendgroup = 'upper', theme = theme, color_theme = color_theme)
# fig_data = add_ma_overlays(fig_data, close_tk, ema_list[: 1], legendgroup = 'upper', theme = theme, color_theme = color_theme_2)

# atr_data = plot_stacked_plotly(close_tk, atr, tk, price_type = 'close', stacked_label = 'MACD', theme = theme)
# atr_fig = atr_data['fig']
# traces_1 = [x for x in atr_fig['data'] if x['legendgroup'] == '1']
# print(len(traces_1))
fig_data = add_macd_stacked(fig_data, tk, macd_data, n_yticks_max = 7)
fig_data = add_ma_overlays(fig_data, close_tk, ma_list[: 6], legendgroup = 'upper', theme = theme, color_theme = color_theme)
macd_fig = fig_data['fig']
n_traces =len([x for x in macd_fig['data'] if x['legendgroup'] in ['upper', 'lower']])
print(n_traces)
macd_fig.show()

# atr_data = add_ma_overlays(atr_data, close_tk, ma_list, theme = theme, color_theme = color_theme)
# atr_data = add_ma_overlays(atr_data, close_tk, ma_list, legendgroup = 'upper', theme = theme, color_theme = color_theme)

### Fix y_min, y_max when adding overlays to lower graph

# new_fig = atr_data['fig']|
# print(new_fig['data'])
# traces_1 = [x for x in new_fig['data'] if x['legendgroup'] == '1']
# print(len(traces_1))
# new_fig.show()

2023-09-28 00:00:00
10


In [18]:
print(macd_fig['layout']['margin'])
print(macd_fig['layout']['height'])
print(macd_fig['data']) #['margin'])

# print(macd_fig['layout']['template']['layout']['paper_bgcolor'])
# print(macd_fig['layout']['template']['layout']['plot_bgcolor'])

layout.Margin({
    't': 60
})
1000
(Scatter({
    'legendgroup': 'upper',
    'line': {'color': 'deepskyblue'},
    'name': 'Close',
    'showlegend': True,
    'x': array(['2023-09-28', '2023-09-29', '2023-10-02', ..., '2024-09-25',
                '2024-09-26', '2024-09-27'], dtype=object),
    'xaxis': 'x',
    'y': array([170.69000244, 171.21000671, 173.75      , ..., 226.36999512,
                227.52000427, 227.78999329]),
    'yaxis': 'y'
}), Scatter({
    'legendgroup': 'dummy',
    'line': {'width': 0},
    'showlegend': False,
    'x': array(['2023-09-28', '2023-09-29', '2023-10-02', ..., '2024-09-25',
                '2024-09-26', '2024-09-27'], dtype=object),
    'xaxis': 'x2',
    'y': array([170.69000244, 171.21000671, 173.75      , ..., 226.36999512,
                227.52000427, 227.78999329]),
    'yaxis': 'y2'
}), Scatter({
    'legendgroup': 'upper',
    'line': {'color': 'rgb(255, 165, 220)'},
    'name': 'SMA 5',
    'showlegend': True,
    'x': array(['2023-09-

Test Bollinger Band Overlays

Candlestick

In [19]:
def plot_candlestick_plotly(
    df_ohlc,
    tk,
    candle_type = 'hollow',
    n_ticks_max = 48,
    n_yticks_max = 16,
    plot_width = 1450,
    plot_height = 750,
    title_font_size = 32,
    theme = 'dark'
):
    """
    candle_type: 'hollow' or 'traditional'
        
    """
    
    style = theme_style[theme]
    red_color = style['red_color']
    green_color = style['green_color']

    df = df_ohlc.copy()

    min_y = min(df['Low'])
    max_y = max(df['High'])
    y_min, y_max = set_axis_limits(min_y, max_y)

    df['Date'] = df.index.astype(str)
    x_min = df['Date'].min()
    x_max = df['Date'].max()

    fig = make_subplots(rows = 1, cols = 1)

    if candle_type == 'traditional':
        
        title = f'{tk} Prices - Traditional Candles'

        shown_green = False
        shown_red = False

        for _, row in df.iterrows():

            if row['Close'] >= row['Open']:
                color_dict = dict(
                    fillcolor = 'rgba(0, 255, 0, 0.3)',
                    line = dict(color = green_color)
                )
                name = 'Close > Open'
                current_candle = 'green'
            else:
                color_dict = dict(
                    fillcolor = 'rgba(255, 0, 0, 0.6)',
                    line = dict(color = red_color)
                )
                name = 'Open > Close'
                current_candle = 'red'

            # Make sure each candle type appears only once in the legend
            if (not shown_green) & (current_candle == 'green'):
                showlegend = True
                shown_green = True
            elif (not shown_red) & (current_candle == 'red'):
                showlegend = True
                shown_red = True
            else:
                showlegend = False

            fig.add_trace(
                go.Candlestick(
                    x = [row['Date']],
                    open = [row['Open']],
                    high = [row['High']],
                    low = [row['Low']],
                    close = [row['Close']],
                    name = name,
                    increasing = color_dict,
                    decreasing = color_dict,
                    showlegend = showlegend,
                    legendgroup = 'upper'
                )
            )
        
    else:  # candle_type == 'hollow'
        
        title = f'{tk} Prices - Hollow Candles'
        
        df['previousClose'] = df['Close'].shift(1)
        
        # Define color based on close and previous close
        df['color'] = np.where(df['Close'] > df['previousClose'], green_color, red_color)
        
        # Set fill to transparent if close > open and the previously defined color otherwise
        df['fill'] = np.where(df['Close'] > df['Open'], 'rgba(255, 0, 0, 0)', df['color'])
        
        shown_red_fill = False
        shown_red_hollow = False
        shown_green_fill = False
        shown_green_hollow = False
        
        for _, row in df.iterrows():
            
            if (row['color'] == green_color) & (row['fill'] == green_color):
                name = 'Open > Close > Prev Close'
                current_candle = 'green_fill'
            elif (row['color'] == green_color) & (row['fill'] == 'rgba(255, 0, 0, 0)'):
                name = 'Prev Close < Close > Open'
                current_candle = 'green_hollow'
            elif (row['color'] == red_color) & (row['fill'] == red_color):
                name = 'Open > Close < Prev Close'
                current_candle = 'red_fill'
            elif (row['color'] == red_color) & (row['fill'] == 'rgba(255, 0, 0, 0)'):
                name = 'Prev Close > Close > Open'
                current_candle = 'red_hollow'
            else:
                name = 'Hollow Candles'
            
            # Make sure each candle type appears only once in the legend
            if (not shown_green_fill) & (current_candle == 'green_fill'):
                showlegend = True
                shown_green_fill = True
            elif (not shown_green_hollow) & (current_candle == 'green_hollow'):
                showlegend = True
                shown_green_hollow = True
            elif (not shown_red_fill) & (current_candle == 'red_fill'):
                showlegend = True
                shown_red_fill = True
            elif (not shown_red_hollow) & (current_candle == 'red_hollow'):
                showlegend = True
                shown_red_hollow = True
            else:
                showlegend = False
        
            color_dict = dict(
                fillcolor = row['fill'],
                line=dict(color = row['color'])
            )
            
            fig.add_trace(
                go.Candlestick(
                    x = [row['Date']],
                    open = [row['Open']],
                    high = [row['High']],
                    low = [row['Low']],
                    close = [row['Close']],
                    increasing = color_dict,
                    decreasing = color_dict,
                    showlegend = showlegend,
                    legendgroup = 'upper',
                    name = name
                )
            )
        
    # Add plot border
    fig.add_shape(
        type = 'rect',
        xref = 'x',  # use 'x' to avoid double lines at x_min and x_max
        yref = 'paper',
        x0 = x_min,
        x1 = x_max,
        y0 = 0,
        y1 = 1,
        line_color = style['x_linecolor'],
        line_width = 0.3
    )
    # Update layout and axes
    fig.update_layout(
        width = plot_width,
        height = plot_height,
        xaxis_rangeslider_visible = False,
        template = style['template'],
        legend_groupclick = 'toggleitem',
        yaxis_title = f'Price',
        margin_t = 60,
        title = dict(
            text = title,
            font_size = title_font_size,
            y = 0.975,
            x = 0.45,
            xanchor = 'center',
            yanchor = 'top'
        )
    )
    fig.update_xaxes(
        type = 'category',
        gridcolor = style['x_gridcolor'],        
        nticks = n_ticks_max,
        tickangle = -90,
        ticks = 'outside',
        ticklen = 8
    )
    fig.update_yaxes(
        range = (y_min, y_max),
        gridcolor = style['y_gridcolor'],
        nticks = n_yticks_max,
        ticks = 'outside',
        ticklen = 8
    )

    fig_data = {
        'fig': fig,
        'y_min': y_min,
        'y_max': y_max
    }

    return fig_data

In [20]:
# Traditional candle patterns:
# https://www.incrediblecharts.com/candlestick_patterns/candlestick-patterns.php

x_min = datetime(2024, 6, 21)
x_max = datetime(2024, 9, 19)

theme = 'dark'
color_theme = 'gold'

candle_data = plot_candlestick_plotly(df_ohlc[x_min: x_max], tk, candle_type = 'traditional', theme = theme)

candle_fig = candle_data['fig']

candle_fig.show()

In [21]:
price_list = [
    {
        'name': 'Adjusted Close',
        'data': adj_close_tk,
        'show': False
    },
    {
        'name': 'Open',
        'data': ohlc_tk['Open'],
        'show': True
    },
    {
        'name': 'Close',
        'data': close_tk,
        'show': True
    },
    {
        'name': 'Low',
        'data': ohlc_tk['Low'],
        'show': False
    },
    {
        'name': 'High',
        'data': ohlc_tk['High'],
        'show': False
    }
]

In [22]:
def add_price_overlays(
    fig_data,
    price_list,
    x_min = None,
    x_max = None,
    legendgroup = 'upper',
    theme = 'dark',
    color_theme = 'gold'
):
    """
    fig_data:
        A dictionary containing the underlying figure data
    legendgroup:
        'upper' is top graph in subplots (row 1), 'lower' is the stacked lower graph (row 2)                
    price_list: 
        list of dictionaries with keys
         - 'name': 'Adjusted Close', 'Close', 'Open', 'High', and 'Low'
         - 'show': True / False - include in plot or not
    x_min, x_max:
        minimum and maximum dates in the datetime format
    """
    
    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max

    # Count lines that will be overlaid ('show' is True)
    # n_price = sum(x.get('show') for x in price_list)

    selected_prices = [x for x in price_list if x['show']]
    n_price = len(selected_prices)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_price]

    current_names = [trace['name'] for trace in fig_data['fig']['data']]
    
    price_overlays = []

    for i, price in enumerate(selected_prices):
        
        price_name = price['name']
    
        if price_name not in current_names:

            price_data = price['data'][x_min: x_max]
            color_idx = overlay_color_idx[i]

            price_overlays.append({
                'data': price_data,
                'name': price_name,
                'color_idx': color_idx
            })

    color_map = {}

    for overlay in price_overlays:
        fig_data = add_overlay(
            fig_data,
            overlay['data'],
            overlay['name'],
            overlay['color_idx'],
            legendgroup = legendgroup,
            theme = theme,
            color_theme = color_theme
        )        
        color_map.update({overlay['name']: overlay['color_idx']})

    fig_data.update({'color_map': color_map})

    return fig_data

In [23]:
x_min = datetime(2024, 6, 21)
x_max = datetime(2024, 9, 19)

theme = 'dark'
color_theme = 'sapphire'

# candle_data = plot_candlestick_plotly(df_ohlc[x_min: x_max], tk, candle_type = 'traditional', theme = theme)
candle_data = plot_candlestick_plotly(df_ohlc[x_min: x_max], tk, candle_type = 'hollow', theme = theme)
# candle_data = add_price_overlays(candle_data, price_list, x_min, x_max, theme = theme, color_theme = color_theme)

candle_fig = candle_data['fig']
candle_fig.show()

In [24]:
def bollinger_bands(
    prices,
    window = 20,
    n_std = 2.0,
    n_bands = 1
):
    """
    prices:
        series of ticker prices ('adjusted close', 'open', 'high', 'low' or 'close')
    window:
        size of the rolling window in days, defaults to 20
    n_std:
        width of the upper and lower bands in standard deviations, defaults to 2.0
    n_bands:
        number of pairs of bands to be created, defaults to 1, max 3

    Returns a list of bollinger band dictionaries
    """

    eps = 1e-6

    n_bands = min(3, n_bands)

    df_sma = prices.rolling(window = window, min_periods = 1).mean()
    df_std = prices.rolling(window = window, min_periods = 1).std(ddof=0)
    
    bollinger_list = [{
        'data': df_sma,
        'name': f'SMA {window}',
        'idx_offset': 0,
        'showlegend': True
    }]

    k = 0
    # k = 0 if each band_width is an integer within the accuray of eps    
    for i in range(n_bands + 1)[1:]:
        band_width = i * n_std
        if abs(float(int(band_width)) - band_width) > eps:
            k = 1
            break

    for i in range(n_bands + 1)[1:]:
        
        band_width = i * n_std

        upper_band = df_sma + band_width * df_std
        upper_name = f'({window}, {band_width:.{k}f}) Upper Bollinger'
        bollinger_list.append({
            'data': upper_band,
            'name': upper_name,
            'idx_offset': i,
            'showlegend': True
        })

        lower_band = df_sma - band_width * df_std        
        lower_name = f'({window}, {band_width:.{k}f}) Lower Bollinger'
        bollinger_list.append({
            'data': lower_band,
            'name': lower_name,
            'idx_offset': -i,
            'showlegend': True
        })

    bollinger_list = sorted(bollinger_list, key = itemgetter('idx_offset'), reverse = True)

    return bollinger_list

In [25]:
def add_bollinger_overlays(
    fig_data,
    bollinger_list,
    x_min = None,
    x_max = None,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    df_price: df_close or df_adj_close, depending on the underlying figure in fig_data

    """

    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max

    n_boll = int((len(bollinger_list) + 1) / 2)
    print(n_boll)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_boll]
    
    current_names = [trace['name'] for trace in fig_data['fig']['data']]

    bollinger_overlays = []
    
    for boll in bollinger_list:
        
        if boll['name'] not in current_names:
            bollinger_overlays.append({
                'data': boll['data'][x_min: x_max],
                'name': boll['name'],
                'color_idx': overlay_color_idx[abs(boll['idx_offset'])],
                'showlegend': boll['showlegend']
            })

    color_map = {}

    for overlay in bollinger_overlays:
        fig_data = add_overlay(
            fig_data,
            overlay['data'],
            overlay['name'],
            overlay['color_idx'],
            overlay['showlegend'],
            theme = theme,
            color_theme = color_theme
        )
        color_map.update({overlay['name']: overlay['color_idx']})

    fig_data.update({'color_map': color_map})

    return fig_data

In [26]:
theme = 'light'
theme = 'dark'
color_theme = 'gold'
color_theme = 'gold'

bollinger_list = bollinger_bands(df_adj_close[tk], 20, 2, 3)
fig_hist_data = plot_hist_plotly(df_adj_close, tk, theme = theme)
fig_boll_new = add_bollinger_overlays(fig_hist_data, bollinger_list, theme = theme, color_theme = color_theme)

fig_boll_new['fig'].show()

4


In [27]:
def ma_envelopes(
    prices,
    ma_type = None,
    window = 20,
    prc_offset = 5,
    n_bands = 3
):
    """
    prices:
        series of ticker prices ('adjusted close', 'open', 'high', 'low' or 'close')
    ma_type:
        one of 'sma', 'ema', dema', tema'
    window:
        size of the rolling window in days
    prc_offset: 
        vertical offset from base moving average in percentage points (-99% to 99%)
    n_bands:
        number of pairs of envelopes to be created, defaults to 3 (max)

    Returns a list of ma envelope dictionaries
    """

    eps = 1e-6
    
    if ma_type is None:
        ma_type = 'sma'

    n_bands = min(3, n_bands)
    if abs(prc_offset) > 99:
        prc_offset = math.sign(prc_offset) * 99

    base_ma = moving_average(prices, ma_type, window)

    base_name = f'{ma_type.upper()} {window}'
    
    ma_envelope_list = [{
        'data': base_ma,
        'name': base_name,
        'idx_offset': 0,
        'showlegend': True
    }]

    k = 0
    # k = 0 if each ma_offset is an integer within the accuray of eps    
    for i in range(n_bands + 1)[1:]:
        ma_offset = i * prc_offset
        if abs(float(int(ma_offset)) - ma_offset) > eps:
            k = 1
            break

    for i in range(n_bands + 1)[1:]:
        
        ma_offset = i * prc_offset

        upper_band = base_ma * (1 + ma_offset / 100)
        upper_name = f'({window}, {ma_offset:.{k}f}%) Upper Envelope'
        ma_envelope_list.append({
            'data': upper_band,
            'name': upper_name,
            'idx_offset': i,
            'showlegend': True
        })

        lower_band = base_ma * (1 - ma_offset / 100)
        lower_name = f'({window}, {ma_offset:.{k}f}%) Lower Envelope'
        ma_envelope_list.append({
            'data': lower_band,
            'name': lower_name,
            'idx_offset': -i,
            'showlegend': True
        })

    ma_envelope_list = sorted(ma_envelope_list, key = itemgetter('idx_offset'), reverse = True)

    return ma_envelope_list

In [28]:
def add_ma_envelope_overlays(
    fig_data,
    ma_envelope_list,
    x_min = None,
    x_max = None,
    theme = 'dark',
    color_theme = 'gold'
):
    """
    df_price: df_close or df_adj_close, depending on the underlying figure in fig_data

    """
    
    x_min = start_date if x_min is None else x_min
    x_max = end_date if x_max is None else x_max
    
    n_env = int((len(ma_envelope_list) + 1) / 2)

    style = theme_style[theme]
    overlay_color_idx = style['overlay_color_selection'][color_theme][n_env]
    
    current_names = [trace['name'] for trace in fig_data['fig']['data']]

    ma_envelope_overlays = []
    
    for env in ma_envelope_list:
        
        if env['name'] not in current_names:
            ma_envelope_overlays.append({
               'data': env['data'][x_min: x_max],
               'name': env['name'],
               'color_idx': overlay_color_idx[abs(env['idx_offset'])],
               'showlegend': env['showlegend']
            })

    color_map = {}

    for overlay in ma_envelope_overlays:
        fig_data = add_overlay(
            fig_data,
            overlay['data'],
            overlay['name'],
            overlay['color_idx'],
            overlay['showlegend'],
            theme = theme,
            color_theme = color_theme
        )
        color_map.update({overlay['name']: overlay['color_idx']})

    fig_data.update({'color_map': color_map})

    return fig_data

In [29]:
theme = 'light'
theme = 'dark'
# color_theme = 'rainbow'

fig_hist_data = plot_hist_plotly(df_adj_close, tk, theme = theme)

# ma_type = 'sma'
window = 50
prc_offset = 2.5
n_bands = 3
color_theme = 'gold'

ma_envelope_list = ma_envelopes(df_adj_close[tk], window = window, prc_offset = prc_offset, n_bands = n_bands)
fig_ma_env = add_ma_envelope_overlays(fig_hist_data, ma_envelope_list, theme = theme, color_theme = color_theme)
fig_ma_env['fig'].show()

window = 10
prc_offset = 2.5
n_bands = 3
color_theme = 'magenta'

ma_envelope_list = ma_envelopes(df_adj_close[tk], window = window, prc_offset = prc_offset, n_bands = n_bands)
fig_ma_env = add_ma_envelope_overlays(fig_hist_data, ma_envelope_list, theme = theme, color_theme = color_theme)
fig_ma_env['fig'].show()

# names = [x['name'] for x in fig_ma_env['fig']['data']]
# print(names)

# bollinger_list = bollinger_bands(df_adj_close[tk], 20, 2, 2)
# fig_boll_new = add_bollinger_overlays(fig_ma_env, bollinger_list, theme = theme, color_theme = 'turquoise')

# fig_boll_new['fig'].show()

In [30]:
from analyze_prices import AnalyzePrices
from mapping_plot_attributes import theme_style

analyze_prices = AnalyzePrices(end_date, start_date, [tk])
theme = 'dark'
color_theme = 'turquoise'
style= theme_style[theme]

# candle_data = plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = 'dark')
# candle_data = analyze_prices.plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = 'dark')
candle_data = plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = 'dark')
candle_fig = candle_data['fig']

bollinger_list = bollinger_bands(df_close[tk], 20, 2, 1)
# candle_data = analyze_prices.add_bollinger_overlays(candle_data, bollinger_list, theme = theme, color_theme = 'sapphire')
candle_data = add_bollinger_overlays(candle_data, bollinger_list, theme = theme, color_theme = 'sapphire')

fig_boll = candle_data['fig']

# fig_boll.show()

window = 20
prc_offset = 10
n_bands = 1

# ma_envelope_list = ma_envelopes(df_adj_close[tk], ma_type, window, prc_offset = prc_offset, n_bands = n_bands)
ma_envelope_list = ma_envelopes(df_close[tk], window = window, prc_offset = prc_offset, n_bands = n_bands)
candle_data = add_ma_envelope_overlays(candle_data, ma_envelope_list, theme = theme, color_theme = 'turquoise')
fig_ma_env = candle_data['fig']
fig_ma_env.show()

2


Moving Average Overlays

In [31]:
ma_list = [
    {
        'ma_idx': 1,
        'ma_type': 'sma',
        # 'ma_window': 10,
        'ma_window': 5,
        'showlegend': True
    },
    {
        'ma_idx': 2,
        'ma_type': 'sma',
        # 'ma_window': 20,
        'ma_window': 10,
        'showlegend': True
    },
    {
        'ma_idx': 3,
        'ma_type': 'sma',
        # 'ma_window': 30,
        'ma_window': 15,
        'showlegend': True
    },
    {
        'ma_idx': 4,
        'ma_type': 'sma',
        # 'ma_window': 40,
        'ma_window': 20,
        'showlegend': True
    }
    ,
    {
        'ma_idx': 5,
        'ma_type': 'sma',
        #'ma_window': 50,
        'ma_window': 25,
        'showlegend': True
    },
    {
        'ma_idx': 6,
        'ma_type': 'sma',
        # 'ma_window': 60,
        'ma_window': 30,
        'showlegend': True
    }
]


In [32]:
def update_color_theme(
    fig,
    color_map,
    theme,
    color_theme,
    invert = False
):
    """
    fig = fig_data['fig']
    color_map = fig_data['color_map']: an overlay color map dictionary
    theme: existing theme ('dark' or light')
    color_theme: new color theme to apply to overlays in fig
    invert: invert the palette from lightest-darkest to darkest-lightest or vice versa?

    Returns updated fig
    """
    
    style = theme_style[theme]
    overlay_colors = style['overlay_color_theme'][color_theme]

    for name, color_idx in color_map.items():
        
        if invert:
            color_idx = len(color_map) - color_idx - 1

        fig.update_traces(
            line_color = overlay_colors[color_idx],
            selector = dict(name = name)
        )
    
    return fig

In [33]:
theme = 'light'
# theme = 'dark'

color_theme = 'lavender'

from mapping_plot_attributes import theme_style
from analyze_prices import AnalyzePrices
analyze_prices = AnalyzePrices(end_date, start_date, [tk])

x_min = datetime(2023, 9, 20)
x_max = datetime(2024, 9, 19)

candle_data = plot_candlestick_plotly(df_ohlc[x_min: x_max], tk, candle_type = 'traditional', theme = theme)
# candle_data = analyze_prices.plot_candlestick_plotly(df_ohlc, tk, candle_type = 'traditional', theme = theme)
candle_data = add_ma_overlays(candle_data, close_tk[x_min: x_max], ma_list, theme = theme, color_theme = color_theme)
# candle_data = add_ma_overlays(candle_data, close_tk, ma_list, theme = theme, color_theme = color_theme)

candle_fig = update_color_theme(
    candle_data['fig'],
    candle_data['color_map'],
    theme = theme,
    color_theme = color_theme,
    invert = True
)
candle_fig.show()

In [34]:
from mapping_plot_attributes import theme_style
from analyze_prices import AnalyzePrices
analyze_prices = AnalyzePrices(end_date, start_date, [tk])

theme = 'light'
# theme = 'dark'
color_theme = 'magenta'

fig_hist_data = plot_hist_plotly(df_adj_close, tk, theme = theme)
fig_hist = fig_hist_data['fig']

new_data = add_ma_overlays(fig_hist_data, df_adj_close[tk], ma_list, theme = theme, color_theme = color_theme)
new_data_fig = new_data['fig']
new_data_fig.show()

In [35]:
print(new_data_fig['data'])

(Scatter({
    'legendgroup': 'upper',
    'line': {'color': '#1f77b4'},
    'name': 'Adjusted Close',
    'showlegend': True,
    'x': array(['2023-09-28', '2023-09-29', '2023-10-02', ..., '2024-09-25',
                '2024-09-26', '2024-09-27'], dtype=object),
    'y': array([169.8210144 , 170.33837891, 172.86543274, ..., 226.36999512,
                227.52000427, 227.78999329])
}), Scatter({
    'legendgroup': 'upper',
    'line': {'color': 'rgb(255, 130, 206)'},
    'name': 'SMA 5',
    'showlegend': True,
    'x': array(['2023-09-28', '2023-09-29', '2023-10-02', ..., '2024-09-25',
                '2024-09-26', '2024-09-27'], dtype=object),
    'xaxis': 'x',
    'y': array([169.8210144 , 170.07969666, 171.00827535, ..., 227.4559967 ,
                227.18599854, 227.1039978 ]),
    'yaxis': 'y'
}), Scatter({
    'legendgroup': 'upper',
    'line': {'color': 'rgb(253, 100, 194)'},
    'name': 'SMA 10',
    'showlegend': True,
    'x': array(['2023-09-28', '2023-09-29', '2023-10-0

In [36]:
print(new_data['color_map'])

{'SMA 5': 5, 'SMA 10': 4, 'SMA 15': 3, 'SMA 20': 2, 'SMA 25': 1, 'SMA 30': 0}


In [37]:
# https://plotly.com/python/creating-and-updating-figures/

from mapping_plot_attributes import theme_style

theme = 'light'
# theme = 'dark'
style= theme_style[theme]
new_color_theme = 'tableau'

new_data_fig = update_color_theme(new_data_fig, new_data['color_map'], theme, new_color_theme, invert = True)
# new_data_fig = update_color_theme(new_data_fig, new_data['color_map'], theme, new_color_theme)

new_data_fig.show()